In [103]:
#%reset
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.pyplot import figure
import matplotlib.transforms as transforms
from scipy import integrate
import scipy

SMALL_SIZE = 10
matplotlib.rc('font',size = SMALL_SIZE, family = 'Arial')
matplotlib.rc('axes',titlesize = SMALL_SIZE)


#define distance to sun/star
a = np.linspace(0.1, 6.0, 100) #[AU]

hfont = {'fontname':'Helvetica'}



#define constants 
mu = 2.34
m_p = 1.67*10**(-24) #[g]
k_B = 1.38*10**(-16)#[erg/K]
G = 6.67408 * 10**(-8) #[dyne cm^2/g^2]
M_mars = 0.64171*10**(27) #[g]
M_sun = 1.989*10**(33) #[g]
M_earth = 5.98*10**27 #[g]
R_E = 6.378*10**8 #[cm]
autocm = 1.496*10**(13)

#define parameters of Mars
P_bot = 10**6 #[Ba] 
R_bot = 3.38*10**8 #[cm] https://www.sciencedirect.com/topics/physics-and-astronomy/planetary-cores

T_0 = 1000

#define temperature
def T_disk(a):
    return T_0*(a/0.1)**(-3/7) #[K]

def Cs_disk(a):
    return np.sqrt(k_B*T_disk(a)/mu/m_p) #CHECK UNITS [cm/s]

def Par(a, M):  
    #compute the R_out based on Boundy/Hill Radisu
    R_H = a*(M/3/M_sun)**(1/3)*autocm #cm
    R_B = G*M/Cs_disk(a)**2 #cm
    
    if type(a) is float:
        R_out = [0]
        R_out = min(R_H,R_B)
        R_out_E = R_out/R_E
    else:
        R_out = [0]*len(R_H)
        for i in range(len(R_H)):
            R_out[i] = min(R_B[i],R_H[i])
        #Bondi Radius always smaller
        R_out_E = [i/R_E for i in R_out]
    print(R_out)
    
    A = G*M/Cs_disk(a)**2 #CHECK UNITS [cm]


    exp = np.exp(-A/R_out + A/R_bot)
    rho_bot = P_bot/Cs_disk(a)**2 #[g/cm^3]
    rho_ne = rho_bot/exp
    
    return A, rho_bot, R_out, rho_ne


def M_int(x):
    A, rho_bot, R_mars, rho_ne= Par(1.5, M_mars)
    print(f'Rho_ne is :{rho_ne}')
    print(f'Ratio is: {rho_ne/(6*10**(-6)*15**(-2.9))}')
    return 4*np.pi*rho_ne*(x**2*np.exp(A/R_mars - A/x))


x = np.linspace(R_bot*1.001, R_out, 1000) # check to change to log spacing
print(f'Outer Boundary is: {R_out}')
print(f'The core radius is: {R_bot}')
y1 = M_int(x)
                         
M = integrate.simpson(y1, x)
GCR = M/M_mars
print(f'The Atmospheric mass is: {M}g')
print(f'Mass core: {M_mars}g')
print(f'GCR is: {M/M_mars}')
print(M_mars/M_earth)


Outer Boundary is: 9677486564.73533
The core radius is: 338000000.0
3870994625.894134
Rho_ne is :2.609549366104725e-09
Ratio is: 1.1196413208207685
The Atmospheric mass is: 1.5280228998799239e+22g
Mass core: 6.4171e+26g
GCR is: 2.381173582895582e-05
0.10730936454849498


In [123]:
kappa = 1
rho = GCR*M_mars
T_rcb = 1000*(1.5 /0.1)**(-3/7)
A, rho_bot, R_mars, rho_ne= Par(1.5, M_mars)
rho_MMEN = 6*10**(-6)*(1.5/0.1)**(-2.9)
print(f'rho MMEN is:{rho_MMEN}')
print(f'ratio of rho is:{rho_ne/rho_MMEN}')
print(f'Temperature is: {T_rcb:.3e}K')
Dad = 0.25
sb = 5.670374 * 10**(-5)
L = 64*np.pi*G*(1+GCR)*M_mars*sb*T_rcb**3*mu*m_p*Dad/(3*k_B*rho_ne*kappa)
print(f'The Cooling Luminosity is: {L:.3e} erg/s')
print(f'The outer boundary is: {R_mars/R_E}')

3870994625.894134
rho MMEN is:2.3307011965151114e-09
ratio of rho is:1.1196413208207685
Temperature is: 3.133e+02K
The Cooling Luminosity is: 1.358e+25 erg/s
The outer boundary is: 6.069292295224418


In [131]:
kappa = 1

T_rcb = 2500
A, rho_bot, R_mars, rho_ne= Par(1.5, M_mars)
rho = GCR*M_mars/(4*np.pi*R_mars**3)
rho_MMEN = 6*10**(-6)*(1.5/0.1)**(-2.9)
print(f'rho MMEN is:{rho_MMEN}')
print(f'ratio of rho is:{rho_ne/rho_MMEN}')
print(f'Temperature is: {T_rcb:.3e}K')
Dad = 0.25
sb = 5.670374 * 10**(-5)
print(f'rho ne is: {rho_ne}')
L = 64*np.pi*G*(1+GCR)*M_mars*sb*T_rcb**3*mu*m_p*Dad/(3*k_B*rho*kappa)
print(f'The Cooling Luminosity is: {L:.3e} erg/s')
print(f'The outer boundary is: {R_mars/R_E}')

3870994625.894134
rho MMEN is:2.3307011965151114e-09
ratio of rho is:1.1196413208207685
Temperature is: 2.500e+03K
rho ne is: 2.609549366104725e-09
The Cooling Luminosity is: 4.090e+28 erg/s
The outer boundary is: 6.069292295224418
